In [1]:
import torch
import torchtext
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer            # 分词工具
from torchtext.vocab import build_vocab_from_iterator     # 创建词表工具
from torchtext.data.functional import to_map_style_dataset

C:\ProgramData\Miniconda3\envs\pth111\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.__version__

'1.11.0+cpu'

In [3]:
torchtext.__version__

'0.12.0'

# TorchText文本分类数据集

In [5]:
train_iter, test_iter = torchtext.datasets.IMDB()

In [6]:
tokenizer = get_tokenizer('basic_english')   
def yield_tokens(data):
    for _, text in data:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [7]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x == 'pos')

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        precess_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(precess_text)
    label_list = torch.tensor(label_list)
    text_list = torch.nn.utils.rnn.pad_sequence(text_list)
    return label_list.to(device), text_list.to(device)

In [13]:
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)

In [14]:
train_dataloader = DataLoader(train_dataset, batch_size=64, 
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=64, 
                             shuffle=True, collate_fn=collate_batch)

In [15]:
for i, (l, b) in enumerate(train_dataloader):
    print(l.size(), b.size())
    if i>9:torchtext
        break

torch.Size([64]) torch.Size([840, 64])
torch.Size([64]) torch.Size([893, 64])
torch.Size([64]) torch.Size([878, 64])
torch.Size([64]) torch.Size([1205, 64])
torch.Size([64]) torch.Size([1244, 64])
torch.Size([64]) torch.Size([724, 64])
torch.Size([64]) torch.Size([1012, 64])
torch.Size([64]) torch.Size([1093, 64])
torch.Size([64]) torch.Size([1075, 64])
torch.Size([64]) torch.Size([1188, 64])
torch.Size([64]) torch.Size([762, 64])


# 创建模型

In [16]:
vocab_size = len(vocab)

In [17]:
vocab_size

100684

In [18]:
embeding_dim = 300
hidden_size = 128

In [29]:
class RNN_Net(nn.Module):
    def __init__(self, vocab_size, embeding_dim, hidden_size):
        super(RNN_Net, self).__init__()
        self.em = nn.Embedding(vocab_size, embeding_dim)   
        self.rnn = nn.LSTM(embeding_dim, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, 2)

    def forward(self, inputs):
        x = self.em(inputs)
        output, (hidden_state, cell_state) = self.rnn(x)
        x = F.relu(self.fc1(output[-1]))
        x = self.fc2(x)
        return x

In [30]:
model = RNN_Net(vocab_size, embeding_dim, hidden_size).to(device)

In [31]:
loss_fn = nn.CrossEntropyLoss()
from torch.optim import lr_scheduler
optimizer = torch.optim.Adam(model.parameters(), betas=(0.5, 0.5), lr=0.01)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)

In [32]:
def train(dataloader):
    total_acc, total_count, total_loss, = 0, 0, 0
    model.train()
    for label, text in dataloader:
        predicted_label = model(text)
        loss = loss_fn(predicted_label, label)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()*label.size(0)
    return total_loss/total_count, total_acc/total_count

In [33]:
def test(dataloader):
    model.eval()
    total_acc, total_count, total_loss, = 0, 0, 0

    with torch.no_grad():
        for label, text in dataloader:
            predicted_label = model(text)
            loss = loss_fn(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
            total_loss += loss.item()*label.size(0)
    return total_loss/total_count, total_acc/total_count

In [34]:
def fit(epochs, train_dl, test_dl):
    train_loss = []
    train_acc = []
    test_loss = []
    test_acc = []

    for epoch in range(epochs):
        epoch_loss, epoch_acc = train(train_dl)
        epoch_test_loss, epoch_test_acc = test(test_dl)
        train_loss.append(epoch_loss)
        train_acc.append(epoch_acc)
        test_loss.append(epoch_test_loss)
        test_acc.append(epoch_test_acc)
        exp_lr_scheduler.step()
        template = ("epoch:{:2d}, train_loss: {:.5f}, train_acc: {:.1f}% ," 
                    "test_loss: {:.5f}, test_acc: {:.1f}%")
        print(template.format(
              epoch, epoch_loss, epoch_acc*100, epoch_test_loss, epoch_test_acc*100))
    print("Done!")
    
    return train_loss, test_loss, train_acc, test_acc

In [35]:
EPOCHS = 100

In [36]:
train_loss, test_loss, train_acc, test_acc = fit(EPOCHS, train_dataloader, test_dataloader)

epoch: 0, train_loss: 0.69916, train_acc: 50.2% ,test_loss: 0.69378, test_acc: 50.0%
epoch: 1, train_loss: 0.69399, train_acc: 50.8% ,test_loss: 0.69380, test_acc: 50.0%
epoch: 2, train_loss: 0.69348, train_acc: 50.2% ,test_loss: 0.69605, test_acc: 50.0%
epoch: 3, train_loss: 0.69278, train_acc: 49.8% ,test_loss: 0.69515, test_acc: 50.0%
epoch: 4, train_loss: 0.67794, train_acc: 55.2% ,test_loss: 0.66339, test_acc: 62.8%
epoch: 5, train_loss: 0.57792, train_acc: 71.5% ,test_loss: 0.55734, test_acc: 74.4%
epoch: 6, train_loss: 0.47179, train_acc: 80.5% ,test_loss: 0.51183, test_acc: 77.9%
epoch: 7, train_loss: 0.39794, train_acc: 84.5% ,test_loss: 0.51667, test_acc: 78.6%
epoch: 8, train_loss: 0.33833, train_acc: 87.9% ,test_loss: 0.53838, test_acc: 76.9%
epoch: 9, train_loss: 0.31944, train_acc: 88.9% ,test_loss: 0.46782, test_acc: 82.8%
epoch:10, train_loss: 0.28838, train_acc: 90.3% ,test_loss: 0.50602, test_acc: 82.9%
epoch:11, train_loss: 0.25307, train_acc: 91.8% ,test_loss: 0.487